In [ ]:
# zeroth shell
import pandas as pd

electron_temp = 9992.2722969523056
ff_heating_estimator = [  4.89135279e-24,   4.37696370e-24,   3.75869301e-24,
         4.97847160e-24,   4.52158002e-24,   4.21024499e-24,
         3.94991540e-24,   3.72915649e-24,   3.58902110e-24,
         3.40170224e-24,   3.20848519e-24,   3.03540032e-24,
         2.87314722e-24,   2.74328938e-24,   2.61063140e-24,
         2.50640248e-24,   2.38164559e-24,   2.26967531e-24,
         2.24509826e-24,   2.12378192e-24,   2.02063266e-24,
         1.92509873e-24,   1.83070678e-24,   1.77346374e-24]

data_path = "/home/afullard/tardis-chvogl-configs"
bf_heating_coeff = pd.read_csv(f"{data_path}/bound_free_heating_coeff.csv", index_col=0)
stim_recomb_cooling_coeff = pd.read_csv(f"{data_path}/stim_recomb_cooling_coeff.csv", index_col=0)
electron_densities = pd.read_csv(f"{data_path}/thermal_electron_densities.csv", index_col=0)
ion_number_density = pd.read_csv(f"{data_path}/thermal_ion_number_density.csv", index_col=(0, 1))
level_number_density = pd.read_csv(f"{data_path}/thermal_level_number_density.csv", index_col=(0, 1, 2))
phi_lucy = pd.read_csv(f"{data_path}/thermal_phi_lucy.csv", index_col=0)
col_exc_cooling_rate = pd.read_csv(f"{data_path}/col_exc_cooling_rate.csv", index_col=0)
col_deexc_cooling_rate = pd.read_csv(f"{data_path}/col_deexc_heating_rate.csv", index_col=0)

In [2]:
#outputs

fb_cooling_ct = 1.2018593543520837e-06
ff_cooling_ct = 6.941664530316456e-07
coll_ion_cooling_ct = 1.6125333965984259e-07
bf_heating_ct = 1.2809489753862688e-06
ff_heating_ct = 2.3829164962085199e-07
coll_ion_heating_ct = 1.5946196993219911e-07
total_heating_rate_ct = -3.8611749800759465e-07
fractional_heating_rate_ct = -0.036553097452112299

In [3]:
from pathlib import Path

import astropy.units as u

from tardis.io.atom_data import AtomData
from tardis.plasma.equilibrium.rates.heating_cooling_rates import (BoundFreeThermalRates,
    CollisionalBoundThermalRates,
    CollisionalIonizationThermalRates,
    FreeFreeThermalRates)

from tardis.plasma.electron_energy_distribution import (
    ThermalElectronEnergyDistribution,
)

home = str(Path('~').expanduser())
cmfgen_atom_data = AtomData.from_hdf(home+'/tardis-regression-data/atom-data/nlte_atom_data/TestNLTE_He_Ti.h5')
cmfgen_atom_data.prepare_atom_data([1], "macroatom", [], [(1, 0)])


thermal_electron_distribution = ThermalElectronEnergyDistribution(0 * u.erg, electron_temp * u.K, electron_densities.values[0] * u.cm**-3)

/home/afullard/tardis/tardis/__init__.py:24: UserWarning: Astropy is already imported externally. Astropy should be imported after TARDIS.
  warnings.warn(


Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

In [4]:
bf_rates = BoundFreeThermalRates(cmfgen_atom_data.photoionization_data)
bf_heating, bf_cooling = bf_rates.solve(
    level_number_density,
    ion_number_density,
    thermal_electron_distribution,
    phi_lucy,
    bound_free_heating_estimator=bf_heating_coeff,
    stimulated_recombination_estimator=stim_recomb_cooling_coeff,
)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mtuple index out of range[0m
[0m[1mDuring: typing of static-get-item at /home/afullard/tardis/tardis/transport/montecarlo/estimators/util.py (61)[0m
[1m
File "../../../../tardis/transport/montecarlo/estimators/util.py", line 61:[0m
[1mdef integrate_array_by_blocks(f, x, block_references):
    <source elided>
    """
[1m    integrated = np.zeros((len(block_references) - 1, f.shape[1]))
[0m    [1m^[0m[0m

[0m[1mDuring: Pass nopython_type_inference[0m

In [5]:
ff_rates = FreeFreeThermalRates()
ff_heating, ff_cooling = ff_rates.solve(
    ff_heating_estimator,
    thermal_electron_distribution,
    ion_number_density,
)

In [ ]:
coll_ion_rates = CollisionalIonizationThermalRates(cmfgen_atom_data.photoionization_data)
coll_ion_heating, coll_ion_cooling = coll_ion_rates.solve(
    thermal_electron_distribution.number_density,
    ion_number_density,
    level_number_density,
    collisional_ionization_rate_coefficient,
    phi_lucy,
    )

In [ ]:
coll_bound_rates = CollisionalBoundThermalRates(cmfgen_atom_data.collision_data)
coll_deexc_heating, coll_exc_cooling = coll_bound_rates.solve(
    thermal_electron_distribution.number_density,
    collisional_deexcitation_rate_coefficient,
    collisional_excitation_rate_coefficient,
    level_number_density,
)